In [5]:
import yfinance as yf
from neuralforecast import NeuralForecast
from neuralforecast.models import NBEATS
import pandas as pd
from datetime import datetime, timedelta

/Users/johtorr/Repos/llm-local-rag/.venv/lib/python3.12/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
2024-11-12 22:26:39,566	INFO util.py:154 -- Missing packages: ['ipywidgets']. Run `pip install -U ipywidgets`, then restart the notebook server for rich notebook output.
2024-11-12 22:26:41,626	INFO util.py:154 -- Missing packages: ['ipywidgets']. Run `pip install -U ipywidgets`, then restart the notebook server for rich notebook output.


In [33]:
def get_stock_forecast(stock_symbol: str):

    max_encoder_length = 6
    FREQUENCY='D'
    max_prediction_length = 10 # 5 days in the future
    val_size = 10 # 4 Months forecast horizon in validation

    # Download historical stock data from Yahoo Finance
    stock_symbol='TSLA'

    stock_data = yf.download(stock_symbol, period="1y", interval="1d")

    # Prepare data for the model (assuming the model needs certain columns and format)
    stock_data = stock_data.reset_index().rename(columns={"Date": "ds", "Close": "y"})
    stock_data = stock_data[["ds", "y"]].copy()
    stock_data.columns = ["ds", "y"]
    stock_data['unique_id'] = stock_symbol

    models = [
        NBEATS(
            input_size=max_encoder_length,
            h=max_prediction_length,
            max_steps=max_encoder_length,
        ),
    ]

    # Initialize the NEBATS model for forecasting
    nf = NeuralForecast(models=models, freq=FREQUENCY)
    nf.fit(df=stock_data, val_size=val_size, verbose=False)
    future_dates = nf.predict().reset_index()
    future_dates = future_dates.rename(columns={'unique_id':'ticker', 'ds':'date', 'NBEATS':'prediction'})

    # Format the forecast results
    return future_dates


In [34]:
get_stock_forecast('TSLA')

[*********************100%***********************]  1 of 1 completed
INFO:lightning_fabric.utilities.seed:Seed set to 1
/Users/johtorr/Repos/llm-local-rag/.venv/lib/python3.12/site-packages/neuralforecast/common/_base_model.py:345: UserWarning: val_check_steps is greater than max_steps, setting val_check_steps to max_steps.
  warnings.warn(
INFO:pytorch_lightning.utilities.rank_zero:GPU available: True (mps), used: True
INFO:pytorch_lightning.utilities.rank_zero:TPU available: False, using: 0 TPU cores
INFO:pytorch_lightning.utilities.rank_zero:HPU available: False, using: 0 HPUs
INFO:pytorch_lightning.callbacks.model_summary:
  | Name         | Type          | Params | Mode 
-------------------------------------------------------
0 | loss         | MAE           | 0      | train
1 | padder_train | ConstantPad1d | 0      | train
2 | scaler       | TemporalNorm  | 0      | train
3 | blocks       | ModuleList    | 2.4 M  | train
-------------------------------------------------------
2.4

Epoch 5: 100%|██████████| 1/1 [00:00<00:00, 32.57it/s, v_num=7, train_loss_step=18.00, train_loss_epoch=18.00, valid_loss=31.80]

INFO:pytorch_lightning.utilities.rank_zero:`Trainer.fit` stopped: `max_steps=6` reached.


Epoch 5: 100%|██████████| 1/1 [00:00<00:00, 31.28it/s, v_num=7, train_loss_step=18.00, train_loss_epoch=18.00, valid_loss=31.80]

INFO:pytorch_lightning.utilities.rank_zero:GPU available: True (mps), used: True
INFO:pytorch_lightning.utilities.rank_zero:TPU available: False, using: 0 TPU cores
INFO:pytorch_lightning.utilities.rank_zero:HPU available: False, using: 0 HPUs



Predicting DataLoader 0: 100%|██████████| 1/1 [00:00<00:00, 251.96it/s]


/Users/johtorr/Repos/llm-local-rag/.venv/lib/python3.12/site-packages/neuralforecast/core.py:210: FutureWarning: In a future version the predictions will have the id as a column. You can set the `NIXTLA_ID_AS_COL` environment variable to adopt the new behavior and to suppress this warning.
  warnings.warn(


,ticker,date,prediction
0,TSLA,2024-11-13 00:00:00+00:00,339.634186
1,TSLA,2024-11-14 00:00:00+00:00,310.102844
2,TSLA,2024-11-15 00:00:00+00:00,330.443817
3,TSLA,2024-11-16 00:00:00+00:00,336.188782
4,TSLA,2024-11-17 00:00:00+00:00,324.384918
5,TSLA,2024-11-18 00:00:00+00:00,337.611542
6,TSLA,2024-11-19 00:00:00+00:00,338.835876
7,TSLA,2024-11-20 00:00:00+00:00,338.564636
8,TSLA,2024-11-21 00:00:00+00:00,319.931335
9,TSLA,2024-11-22 00:00:00+00:00,333.775208


In [39]:
import requests
import json
import os 

api_key = os.environ['OPENAI_API_KEY']
endpoint = "https://api.openai.com/v1/chat/completions"

headers = {
    "Content-Type": "application/json",
    "Authorization": f"Bearer {api_key}"
}

# Define the function schema for OpenAI function calling
function_schema = {
    "name": "get_stock_forecast",
    "description": "Get stock price forecast for the next five days",
    "parameters": {
        "type": "object",
        "properties": {
            "stock_symbol": {
                "type": "string",
                "description": "The stock symbol to forecast (e.g., 'AAPL', 'MSFT')."
            }
        },
        "required": ["stock_symbol"]
    }
}

# Define the initial user message
messages = [
    {"role": "user", "content": "Get the 5-day stock price forecast for TESLA."}
]

# Prepare the request data with function calling
data = {
    "model": "gpt-4o-mini",  # or any other compatible model
    "messages": messages,
    "functions": [function_schema],
    "function_call": "auto"
}

In [40]:
# Send the request
response = requests.post(endpoint, headers=headers, data=json.dumps(data))

# Check if the response includes a function call request
if response.status_code == 200:
    result = response.json()
    function_call = result["choices"][0].get("message", {}).get("function_call")

    if function_call:
        # If the model requests a function call, extract arguments and call the function
        arguments = json.loads(function_call["arguments"])
        stock_symbol = arguments["stock_symbol"]
        forecast = get_stock_forecast(stock_symbol)
        print(f"Forecast for {stock_symbol}:", forecast)
    else:
        print(result["choices"][0]["message"]["content"])
else:
    print(f"Request failed with status code {response.status_code}: {response.text}")

[*********************100%***********************]  1 of 1 completed
INFO:lightning_fabric.utilities.seed:Seed set to 1
/Users/johtorr/Repos/llm-local-rag/.venv/lib/python3.12/site-packages/neuralforecast/common/_base_model.py:345: UserWarning: val_check_steps is greater than max_steps, setting val_check_steps to max_steps.
  warnings.warn(
INFO:pytorch_lightning.utilities.rank_zero:GPU available: True (mps), used: True
INFO:pytorch_lightning.utilities.rank_zero:TPU available: False, using: 0 TPU cores
INFO:pytorch_lightning.utilities.rank_zero:HPU available: False, using: 0 HPUs
INFO:pytorch_lightning.callbacks.model_summary:
  | Name         | Type          | Params | Mode 
-------------------------------------------------------
0 | loss         | MAE           | 0      | train
1 | padder_train | ConstantPad1d | 0      | train
2 | scaler       | TemporalNorm  | 0      | train
3 | blocks       | ModuleList    | 2.4 M  | train
-------------------------------------------------------
2.4

Epoch 5: 100%|██████████| 1/1 [00:00<00:00, 29.82it/s, v_num=11, train_loss_step=18.00, train_loss_epoch=18.00, valid_loss=31.80]

INFO:pytorch_lightning.utilities.rank_zero:`Trainer.fit` stopped: `max_steps=6` reached.


Epoch 5: 100%|██████████| 1/1 [00:00<00:00, 28.67it/s, v_num=11, train_loss_step=18.00, train_loss_epoch=18.00, valid_loss=31.80]


INFO:pytorch_lightning.utilities.rank_zero:GPU available: True (mps), used: True
INFO:pytorch_lightning.utilities.rank_zero:TPU available: False, using: 0 TPU cores
INFO:pytorch_lightning.utilities.rank_zero:HPU available: False, using: 0 HPUs


Predicting DataLoader 0: 100%|██████████| 1/1 [00:00<00:00, 248.64it/s]
Forecast for TSLA:   ticker                      date  prediction
0   TSLA 2024-11-13 00:00:00+00:00  339.634186
1   TSLA 2024-11-14 00:00:00+00:00  310.102844
2   TSLA 2024-11-15 00:00:00+00:00  330.443817
3   TSLA 2024-11-16 00:00:00+00:00  336.188782
4   TSLA 2024-11-17 00:00:00+00:00  324.384918
5   TSLA 2024-11-18 00:00:00+00:00  337.611542
6   TSLA 2024-11-19 00:00:00+00:00  338.835876
7   TSLA 2024-11-20 00:00:00+00:00  338.564636
8   TSLA 2024-11-21 00:00:00+00:00  319.931335
9   TSLA 2024-11-22 00:00:00+00:00  333.775208


/Users/johtorr/Repos/llm-local-rag/.venv/lib/python3.12/site-packages/neuralforecast/core.py:210: FutureWarning: In a future version the predictions will have the id as a column. You can set the `NIXTLA_ID_AS_COL` environment variable to adopt the new behavior and to suppress this warning.
  warnings.warn(


In [41]:
function_call

{'name': 'get_stock_forecast', 'arguments': '{"stock_symbol":"TSLA"}'}